## Sudoku
__Problem:__

You are given a 9x9 grid which is partially filled with digits with the task to fill the empty spots so that in each row, column and 3x3 subgrids (total of 9 that make up the grid without overlapping) you'll find all digits from 1 to 9 exactly once.

In [143]:
import numpy as np
import itertools

I will work with 3 example Sudokus of different degrees of difficulty. The digit $0$ will stand for a blank spot.

In [219]:
# 48 given numbers
sudoku_1 = [[1, 8, 7, 0, 5, 9, 0, 3, 2],
            [0, 0, 2, 1, 8, 3, 0, 7, 0],
            [3, 0, 4, 0, 0, 7, 9, 1, 0],
            [0, 7, 6, 0, 9, 0, 0, 0, 3],
            [5, 2, 0, 7, 0, 4, 0, 9, 6],
            [8, 0, 0, 0, 2, 0, 4, 5, 0],
            [0, 3, 8, 9, 0, 0, 2, 0, 1],
            [0, 4, 0, 6, 3, 1, 7, 0, 0],
            [6, 9, 0, 8, 7, 0, 3, 4, 5]]

# 32 given numbers
sudoku_2 = [[2, 0, 0, 3, 0, 0, 0, 0, 7],
            [6, 0, 0, 5, 1, 0, 0, 0, 8],
            [0, 0, 0, 8, 0, 0, 0, 0, 9],
            [0, 6, 9, 0, 7, 0, 4, 0, 0],
            [1, 0, 5, 4, 0, 8, 9, 0, 6],
            [0, 0, 8, 0, 9, 0, 1, 3, 0],
            [5, 0, 0, 0, 0, 3, 0, 0, 0],
            [8, 0, 0, 0, 5, 6, 0, 0, 3],
            [9, 0, 0, 0, 0, 2, 0, 0, 4]]

# 24 given numbers
sudoku_3 = [[0, 0, 0, 9, 0, 0, 0, 0, 8],
            [9, 2, 0, 6, 7, 3, 0, 0, 0],
            [1, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 5, 0, 0, 9],
            [4, 6, 0, 0, 0, 0, 0, 3, 5],
            [8, 0, 0, 2, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 1],
            [0, 0, 0, 5, 3, 8, 0, 6, 7],
            [5, 0, 0, 0, 0, 4, 0, 0, 0]]


sudoku_false_duplicate = [[9, 0, 0, 9, 0, 0, 0, 0, 8],
            [9, 2, 0, 6, 7, 3, 0, 0, 0],
            [1, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 5, 0, 0, 9],
            [4, 6, 0, 0, 0, 0, 0, 3, 5],
            [8, 0, 0, 2, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 1],
            [0, 0, 0, 5, 3, 8, 0, 6, 7],
            [5, 0, 0, 0, 0, 4, 0, 0, 0]]

sudoku_false_max = [[900, 0, 0, 9, 0, 0, 0, 0, 8],
            [9, 2, 0, 6, 7, 3, 0, 0, 0],
            [1, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 5, 0, 0, 9],
            [4, 6, 0, 0, 0, 0, 0, 3, 5],
            [8, 0, 0, 2, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 1],
            [0, 0, 0, 5, 3, 8, 0, 6, 7],
            [5, 0, 0, 0, 0, 4, 0, 0, 0]]

sudoku_false_min = [[-900, 0, 0, 9, 0, 0, 0, 0, 8],
            [9, 2, 0, 6, 7, 3, 0, 0, 0],
            [1, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 5, 0, 0, 9],
            [4, 6, 0, 0, 0, 0, 0, 3, 5],
            [8, 0, 0, 2, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 1],
            [0, 0, 0, 5, 3, 8, 0, 6, 7],
            [5, 0, 0, 0, 0, 4, 0, 0, 0]]

print(np.matrix(sudoku_1))
sudoku_list = [sudoku_1, sudoku_2, sudoku_3]

[[1 8 7 0 5 9 0 3 2]
 [0 0 2 1 8 3 0 7 0]
 [3 0 4 0 0 7 9 1 0]
 [0 7 6 0 9 0 0 0 3]
 [5 2 0 7 0 4 0 9 6]
 [8 0 0 0 2 0 4 5 0]
 [0 3 8 9 0 0 2 0 1]
 [0 4 0 6 3 1 7 0 0]
 [6 9 0 8 7 0 3 4 5]]


Before I can start, I need to validate my input sudoku and I also need a validation for my solution. So let's define a function to check for the properties of a classic Sudoku: 9x9 grid, digits between 1 and 9, no duplicate digit in a row, column or the 3x3 subgrids.

In [245]:
def validation(sudoku):
    sudoku = np.matrix(sudoku)
    if sudoku.shape != (9,9):
        print("Expected a 9x9 grid")
        return False
    if sudoku.max() > 9:
        print("There is a value greater than 9 in the Sudoku!")
        return False
    if sudoku.min() < 0:
        print("There is a value less than 0 in the Sudoku!")
        return False
    for i in range(9):
        for digit in range(1,10):
            if sudoku[:,i].tolist().count(digit) > 1:
                return False
            if sudoku[i,:].tolist().count(digit) > 1:
                return False
    for i, j in itertools.product(range(3), range(3)):
        subgrid = list(itertools.chain.from_iterable(sudoku[i*3:i*3+3,j*3:j*3+3].tolist()))
        for digit in range(1,10):
            if subgrid.count(digit) > 1:
                return False
    if sudoku.sum() == 405:
        print("\nThis Sudoku is solved!")
    else:
        print(sudoku)
        print("\nThis Sudoku isn't solved yet.")

Some quick tests for our validation function.

In [246]:
validation(sudoku_1)
validation(sudoku_2)
validation(sudoku_3)

[[1 8 7 0 5 9 0 3 2]
 [0 0 2 1 8 3 0 7 0]
 [3 0 4 0 0 7 9 1 0]
 [0 7 6 0 9 0 0 0 3]
 [5 2 0 7 0 4 0 9 6]
 [8 0 0 0 2 0 4 5 0]
 [0 3 8 9 0 0 2 0 1]
 [0 4 0 6 3 1 7 0 0]
 [6 9 0 8 7 0 3 4 5]]

This Sudoku isn't solved yet.
[[2 0 0 3 0 0 0 0 7]
 [6 0 0 5 1 0 0 0 8]
 [0 0 0 8 0 0 0 0 9]
 [0 6 9 0 7 0 4 0 0]
 [1 0 5 4 0 8 9 0 6]
 [0 0 8 0 9 0 1 3 0]
 [5 0 0 0 0 3 0 0 0]
 [8 0 0 0 5 6 0 0 3]
 [9 0 0 0 0 2 0 0 4]]

This Sudoku isn't solved yet.
[[0 0 0 9 0 0 0 0 8]
 [9 2 0 6 7 3 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 5 0 0 9]
 [4 6 0 0 0 0 0 3 5]
 [8 0 0 2 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 5 3 8 0 6 7]
 [5 0 0 0 0 4 0 0 0]]

This Sudoku isn't solved yet.


In [247]:
validation(sudoku_false_duplicate)

False

In [248]:
validation(sudoku_false_max)

There is a value greater than 9 in the Sudoku!


False

In [249]:
validation(sudoku_false_min)

There is a value less than 0 in the Sudoku!


False

I will use a brute force methode and solve the sudoku by recursion. This works since it's only a 9x9 grid, but the solution will scale very badly for bigger grids.

I'll need a function to make a valid guess for the digit in a blank spot. Then I will loop over all blank spots, make a guess and brute force my way to one solution.

In [250]:
def guess(sudoku, y, x, digit):
    for i in range(9):
        if sudoku[y][i] == digit:
            return False
    for i in range(9):
        if sudoku[i][x] == digit:
            return False
    x_0, y_0 = (x//3)*3, (y//3)*3
    for i in range(3):
        for j in range(3):
            if sudoku[y_0+i][x_0+j] == digit:
                return False
    return True

In [251]:
def solution(sudoku):
    for y in range(9):
        for x in range(9):
            if sudoku[y][x] == 0:
                for digit in range(1,10):
                    if guess(sudoku, y, x, digit):
                        sudoku[y][x] = digit
                        solution(sudoku)
                        sudoku[y][x] = 0
                return
    print(np.matrix(sudoku))
    validation(sudoku)

In [252]:
for i in range(3):
    validation(sudoku_list[i])
    print("\n")
    solution(sudoku_list[i])
    print("\n")

[[1 8 7 0 5 9 0 3 2]
 [0 0 2 1 8 3 0 7 0]
 [3 0 4 0 0 7 9 1 0]
 [0 7 6 0 9 0 0 0 3]
 [5 2 0 7 0 4 0 9 6]
 [8 0 0 0 2 0 4 5 0]
 [0 3 8 9 0 0 2 0 1]
 [0 4 0 6 3 1 7 0 0]
 [6 9 0 8 7 0 3 4 5]]

This Sudoku isn't solved yet.


[[1 8 7 4 5 9 6 3 2]
 [9 6 2 1 8 3 5 7 4]
 [3 5 4 2 6 7 9 1 8]
 [4 7 6 5 9 8 1 2 3]
 [5 2 3 7 1 4 8 9 6]
 [8 1 9 3 2 6 4 5 7]
 [7 3 8 9 4 5 2 6 1]
 [2 4 5 6 3 1 7 8 9]
 [6 9 1 8 7 2 3 4 5]]

This Sudoku is solved!


[[2 0 0 3 0 0 0 0 7]
 [6 0 0 5 1 0 0 0 8]
 [0 0 0 8 0 0 0 0 9]
 [0 6 9 0 7 0 4 0 0]
 [1 0 5 4 0 8 9 0 6]
 [0 0 8 0 9 0 1 3 0]
 [5 0 0 0 0 3 0 0 0]
 [8 0 0 0 5 6 0 0 3]
 [9 0 0 0 0 2 0 0 4]]

This Sudoku isn't solved yet.


[[2 8 1 3 6 9 5 4 7]
 [6 9 7 5 1 4 3 2 8]
 [4 5 3 8 2 7 6 1 9]
 [3 6 9 2 7 1 4 8 5]
 [1 2 5 4 3 8 9 7 6]
 [7 4 8 6 9 5 1 3 2]
 [5 7 2 9 4 3 8 6 1]
 [8 1 4 7 5 6 2 9 3]
 [9 3 6 1 8 2 7 5 4]]

This Sudoku is solved!


[[0 0 0 9 0 0 0 0 8]
 [9 2 0 6 7 3 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 5 0 0 9]
 [4 6 0 0 0 0 0 3 5]
 [8 0 0 2 0 0 0 0

As you can see, it can solve the Sudokus.